## Sucursales bancarias 2020

En este notebook utilizaremos web scraping para extraer el número de sucursales bancarias que había por municipio en el año 2020.  
Obtendremos un DataFrame con la siguiente información:  
   - Y2020
   - Nombre Provincia
   - Codigo Provincia
   - Nombre municipio
   - Código municipio
   - Nº Sucursales Bancarias

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time
import glob

Definimos la página a scrapear y ruta de descarga del chromedriver:

In [49]:
website = 'https://serviciostelematicosext.hacienda.gob.es/SGFAL/CONPREL'

In [50]:
path = '/home/dsc/Downloads/chromedriver'

Habilitar permisos múltiples descargas

In [51]:
option = Options()

option.add_experimental_option(
    "prefs", {"profile.default_content_setting_values.automatic_downloads": 1} #argumento 1: permitir
)

Definimos variable driver y abrimos Google Chrome:

In [205]:
driver = webdriver.Chrome(
    options = option, executable_path = path
)

In [206]:
driver.get(website)

Seleccionamos el año de extracción de los datos:

In [207]:
Fecha = Select(driver.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/main/div[5]/div/select'))

In [208]:
Fecha.select_by_visible_text('2011')

Del despegable, hay que descargar 19 Excels, uno por CCAA y dos ciudades autónomas:

In [12]:
 from selenium.webdriver.common.keys import Keys

In [212]:
CCAA = Select(driver.find_element_by_id('CCAASeleccionado'))
CCAA.select_by_index(2)
time.sleep(3)
CCAA.deselect_all()
time.sleep(3)
CCAA = Select(driver.find_element_by_id('CCAASeleccionado'))
CCAA.select_by_index(3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="CCAASeleccionado"]"}
  (Session info: chrome=106.0.5249.119)


In [169]:
CCAA = Select(driver.find_element_by_id('CCAASeleccionado'))
for i in range(2, 4):
    CCAA.select_by_index(i)
    time.sleep(3)
    CCAA.deselect_by_index(i)
    time.sleep(3)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=106.0.5249.119)


In [204]:
for i in range(2, 4):
    Fecha = Select(driver.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/main/div[5]/div/select'))
    Fecha.select_by_visible_text('2011')
    CCAA = Select(driver.find_element_by_id('CCAASeleccionado'))
    Descargar = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div/div/main/div[7]/div[1]/select/option['+str(i)+']')
    Descargar.click()
    time.sleep(3)

In [110]:
driver.quit()

Generamos un DataFrame concatenando los archivos de cada una de las CCAA y ciudades autónomas.
Descartamos de cada uno de los Excel las primeras filas por contener información no necesaria:

In [ ]:
dfLista = []
ruta = '/home/dsc/Downloads/' 
Excel_files = glob.glob(ruta + "EP2011C01/*.xls")
count = 0

for file in Excel_files:
    with open(file, 'r', encoding='latin1') as f:
        TotalLineas = 0
        for linea in f:
            TotalLineas+= 1
        count += 1
        if count == 1: # primer cvs tiene encabezado
            Inicio = TotalLineas-6
        else: # resto de csv se procesan sin encabezado
            Inicio = TotalLineas-7
           
        df = pd.read_Excel(file, encoding='latin1', skiprows = range(Inicio, TotalLineas), sep = ';', 
            usecols = [' Cód. Provincia ', ' Cód. Municipio ', ' Municipio ', ' Nº de oficinas ']) 
        dfLista.append(df)
        
SucursalesBancarias20 = pd.concat(dfLista, ignore_index = True)

In [111]:
dfLista = []
ruta = '/home/dsc/Downloads/' 
Excel_files = glob.glob(ruta + "/*.xls")
for file in Excel_files:
    print(file)
    xl = pd.ExcelFile(file)
    sheet_names = xl.sheet_names 
    last_sheet = sheet_names[-1]
    df = pd.read_excel(file, skiprows = range(0, 6), sheet_name = last_sheet)
    dfLista.append(df)
df = pd.concat(dfLista, ignore_index = True)

/home/dsc/Downloads/EP2011C01.xls
/home/dsc/Downloads/EP2011C01 (1).xls


In [112]:
df

,Prov,Corp,Tipo,Nombre,Pobla 2010,Estado Inf. (*),Impuestos directos,Impuestos indirectos,"Tasas, precios públicos y otros ingresos",Transferencias corrientes,...,Activos financieros.1,Pasivos financieros.1,Total gastos,Deuda Pública,Servicios públicos básicos,Actuaciones de protección y promoción social,Producción de bienes públicos de carácter preferente,Actuaciones de carácter económico,Actuaciones de carácter general,Total gastos.1
0,4,0,D,Diputación Prov. de Almería ...,NaN,C,7715560.00,6742120.00,9156268.71,1.043422e+08,...,20868000.0,15320000.00,1.661299e+08,19020000.00,4024700.43,52508295.69,9030346.28,23820785.57,57725817.22,1.661299e+08
1,4,1,A,Abla ...,1480.0,C,336000.00,180100.00,206100.00,4.812000e+05,...,0.0,40000.00,1.425850e+06,60000.00,204500.00,213400.00,719650.00,36100.00,192200.00,1.425850e+06
2,4,2,A,Abrucena ...,1391.0,C,197200.00,11000.00,190300.00,5.070000e+05,...,0.0,39000.00,1.105200e+06,58000.00,25000.00,90000.00,432400.00,135000.00,364800.00,1.105200e+06
3,4,3,A,Adra ...,24512.0,C,5559000.00,150000.00,4218000.00,7.917000e+06,...,2500.0,1412000.00,1.892240e+07,1839000.00,7633500.00,4136000.00,1726150.00,1148250.00,2439500.00,1.892240e+07
4,4,4,A,Albánchez ...,823.0,C,157600.00,31600.00,45600.00,3.600410e+05,...,0.0,7200.00,7.102000e+05,13600.00,29000.00,75500.00,0.00,393305.00,198795.00,7.102000e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,41,101,A,Villaverde del Río ...,7544.0,C,1190201.20,129965.05,730393.98,2.002255e+06,...,17000.0,107510.00,7.071221e+06,145110.00,2609959.78,1398562.00,1291163.94,106510.97,1519914.18,7.071221e+06
1554,41,102,A,Viso del Alcor (El) ...,18828.0,C,4018817.69,19441.21,1823041.05,4.947717e+06,...,65000.0,379333.89,1.150634e+07,435713.00,3871456.53,1854165.82,1510518.84,319300.52,3515188.14,1.150634e+07
1555,41,901,A,Cañada Rosal ...,3262.0,C,568000.00,89262.00,314946.59,7.653684e+05,...,6000.0,100005.12,2.383000e+06,133795.78,790765.30,51400.00,974576.87,35849.79,396612.26,2.383000e+06
1556,41,902,A,Isla Mayor ...,5915.0,C,1496000.00,50000.00,384500.00,1.683431e+06,...,15000.0,117914.91,3.656931e+06,221488.64,1191200.99,757759.93,653734.67,34865.58,797881.24,3.656931e+06
